<a href="https://colab.research.google.com/github/pagand/model_optimze_vessel/blob/yimeng/reinforcement_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install d3rlpy==2.0.3
!pip install gym==2.6.0
!pip install -U transformers==4.30.0
!pip install scikit-learn==1.0.2

from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerModel, TimeSeriesTransformerForPrediction
import torch
import gym
import numpy as np
import pickle
import random
from torch.utils.data import Dataset, DataLoader
from google.colab import drive
import d3rlpy
import pandas as pd
drive.mount('/content/drive')

with open('drive/MyDrive/rl_data.pkl', 'rb') as handle:
    rl_data = pickle.load(handle)

ERROR: Could not find a version that satisfies the requirement gym==2.6.0 (from versions: 0.0.2, 0.0.3, 0.0.4, 0.0.5, 0.0.6, 0.0.7, 0.1.0, 0.1.1, 0.1.2, 0.1.3, 0.1.4, 0.1.5, 0.1.6, 0.1.7, 0.2.0, 0.2.1, 0.2.2, 0.2.3, 0.2.4, 0.2.5, 0.2.6, 0.2.7, 0.2.8, 0.2.9, 0.2.10, 0.2.11, 0.2.12, 0.3.0, 0.4.0, 0.4.1, 0.4.2, 0.4.3, 0.4.4, 0.4.5, 0.4.6, 0.4.8, 0.4.9, 0.4.10, 0.5.0, 0.5.1, 0.5.2, 0.5.3, 0.5.4, 0.5.5, 0.5.6, 0.5.7, 0.6.0, 0.7.0, 0.7.1, 0.7.2, 0.7.3, 0.7.4, 0.8.0.dev0, 0.8.0, 0.8.1, 0.8.2, 0.9.0, 0.9.1, 0.9.2, 0.9.3, 0.9.4, 0.9.5, 0.9.6, 0.9.7, 0.10.0, 0.10.1, 0.10.2, 0.10.3, 0.10.4, 0.10.5, 0.10.8, 0.10.9, 0.10.11, 0.11.0, 0.12.0, 0.12.1, 0.12.4, 0.12.5, 0.12.6, 0.13.0, 0.13.1, 0.14.0, 0.15.3, 0.15.4, 0.15.6, 0.15.7, 0.16.0, 0.17.0, 0.17.1, 0.17.2, 0.17.3, 0.18.0, 0.18.3, 0.19.0, 0.20.0, 0.21.0, 0.22.0, 0.23.0, 0.23.1, 0.24.0, 0.24.1, 0.25.0, 0.25.1, 0.25.2, 0.26.0, 0.26.1, 0.26.2)
ERROR: No matching distribution found for gym==2.6.0
Drive already mounted at /content/drive; to attempt to 

In [ ]:
rl_data[1]["observations"][0]

array([ 1.00000000e+00,  1.73800000e+02,  5.78201792e-02, -3.14107591e-02,
        9.99506560e-01,  1.25905353e+00,  4.07498063e-01,  0.00000000e+00,
        0.00000000e+00,  9.31828385e-03,  0.00000000e+00,  2.13797399e-01,
        1.00000000e+00,  1.00000000e+00,  2.50000000e-01,  2.50000000e-01,
        3.74466392e-01,  1.86616923e-01,  4.21229992e-04,  2.32591947e-03])

In [ ]:
import gym
from gym import spaces
import pickle
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import torch
from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerForPrediction


In [ ]:
with open('drive/MyDrive/rl_data.pkl', 'rb') as handle:
    rl_data = pickle.load(handle)

observations = []
rewards = []
actions = []
terminations = []
# for i in range(len(rl_data)):
for i in range(2):
    if i !=0:
        x = rl_data[i]
        obs = x["observations"]
        # new_obs = np.zeros([len(obs-25), 185])
        # for j in range(25, len(obs)):
            # new_obs[j-25] = np.concatenate([obs[j-25:j,[0,1,2,16,17,18,19]].reshape(1,-1), obs[j, 6:16].reshape(1,-1)], axis=1)
        observations.append(obs)
        rewards.append((np.sum(x["rewards"][:, [0,1,3]], axis=1)/3).reshape(-1,1))
        # rewards.append((np.sum(x["rewards"][:, [0,1,3]], axis=1)/3).reshape(-1,1))
        # rewards.append(x["rewards"][:,3].reshape(-1,1))
        terminations.append(x["termination"])
        x["actions"][:, 1] = x["actions"][:, 1]/360
        actions.append(x["actions"])


In [ ]:
from VesselEnvironment import VesselEnvironment
stage1_env = VesselEnvironment(rl_data, "fc_2_final.pt", "longlat_3_final.pt", pickle.load(open('minmax_scaler.pkl', 'rb')), reward_type="mimic")
stage2_env = VesselEnvironment(rl_data, "fc_2_final.pt", "longlat_3_final.pt", pickle.load(open('minmax_scaler.pkl', 'rb')), reward_type="top1")
stage3_env = VesselEnvironment(rl_data, "fc_2_final.pt", "longlat_3_final.pt", pickle.load(open('minmax_scaler.pkl', 'rb')), reward_type="conceptual")

In [ ]:
import d3rlpy
dt = d3rlpy.algos.DecisionTransformerConfig(max_timestep=124, context_size=25, learning_rate=1e-6).create(device="cuda:0")

In [ ]:
observations = np.vstack(observations)
rewards = np.vstack(rewards)
actions = np.vstack(actions)
terminations = np.vstack(terminations)
dataset_stage1 = d3rlpy.dataset.MDPDataset(
    observations=observations,
    actions=actions,
    rewards=rewards,
    terminals=terminations,
)

In [ ]:

# offline training
dt.fit(
   dataset_stage1,
   n_steps=42000,
   n_steps_per_epoch=6000,
   eval_env=stage2_env,
   eval_target_return=0,  # specify target environment return
)

2023-09-08 19:27:59 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float64')], shape=[(20,)]), action_signature=Signature(dtype=[dtype('float64')], shape=[(3,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=3)
2023-09-08 19:27:59 [info     ] Directory is created at d3rlpy_logs/DecisionTransformer_20230908192759
2023-09-08 19:27:59 [debug    ] Building models...
2023-09-08 19:27:59 [debug    ] Models have been built.
2023-09-08 19:27:59 [info     ] Parameters                     params={'observation_shape': [20], 'action_size': 3, 'config': {'type': 'decision_transformer', 'params': {'batch_size': 64, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'context_size': 25, 'learning_rate': 1e-06, 'encoder_factory': {'type': 'default

Epoch 1/7:   0%|          | 0/6000 [00:00<?, ?it/s]

2023-09-08 19:32:15 [info     ] DecisionTransformer_20230908192759: epoch=1 step=6000 epoch=1 metrics={'time_sample_batch': 0.0049977021217346195, 'time_algorithm_update': 0.027134145498275756, 'loss': 0.4076444088711093, 'time_step': 0.03228634715080261, 'environment': -167.17440984095222} step=6000
2023-09-08 19:32:16 [info     ] Model parameters are saved to d3rlpy_logs/DecisionTransformer_20230908192759/model_6000.d3


Epoch 2/7:   0%|          | 0/6000 [00:00<?, ?it/s]

2023-09-08 19:36:31 [info     ] DecisionTransformer_20230908192759: epoch=2 step=12000 epoch=2 metrics={'time_sample_batch': 0.004831509709358216, 'time_algorithm_update': 0.026864187002182006, 'loss': 0.010827966093279733, 'time_step': 0.03182418608665466, 'environment': -131.00553107103207} step=12000
2023-09-08 19:36:31 [info     ] Model parameters are saved to d3rlpy_logs/DecisionTransformer_20230908192759/model_12000.d3


...

2023-09-08 19:49:15 [info     ] DecisionTransformer_20230908192759: epoch=5 step=30000 epoch=5 metrics={'time_sample_batch': 0.004803719361623128, 'time_algorithm_update': 0.026864957769711813, 'loss': 0.00046560075881522305, 'time_step': 0.03179315694173177, 'environment': -172.10749896370928} step=30000
2023-09-08 19:49:15 [info     ] Model parameters are saved to d3rlpy_logs/DecisionTransformer_20230908192759/model_30000.d3


Epoch 6/7:   0%|          | 0/6000 [00:00<?, ?it/s]

2023-09-08 19:53:27 [info     ] DecisionTransformer_20230908192759: epoch=6 step=36000 epoch=6 metrics={'time_sample_batch': 0.004679686784744262, 'time_algorithm_update': 0.026857677896817526, 'loss': 0.00030742931051160364, 'time_step': 0.03165424716472626, 'environment': -151.14089671446834} step=36000
2023-09-08 19:53:27 [info     ] Model parameters are saved to d3rlpy_logs/DecisionTransformer_20230908192759/model_36000.d3


Epoch 7/7:   0%|          | 0/6000 [00:00<?, ?it/s]

2023-09-08 19:57:44 [info     ] DecisionTransformer_20230908192759: epoch=7 step=42000 epoch=7 metrics={'time_sample_batch': 0.00492082154750824, 'time_algorithm_update': 0.026962741017341613, 'loss': 0.00022622339225684604, 'time_step': 0.03200147414207458, 'environment': -119.61825017351002} step=42000
2023-09-08 19:57:44 [info     ] Model parameters are saved to d3rlpy_logs/DecisionTransformer_20230908192759/model_42000.d3


In [ ]:
observations = np.vstack(observations)
rewards = np.vstack(rewards)
actions = np.vstack(actions)
terminations = np.vstack(terminations)
dataset_stage2 = d3rlpy.dataset.MDPDataset(
    observations=observations,
    actions=actions,
    rewards=rewards,
    terminals=terminations,
)

In [ ]:

# offline training
dt.fit(
   dataset_stage2,
   n_steps=42000,
   n_steps_per_epoch=6000,
   eval_env=stage2_env,
   eval_target_return=0,  # specify target environment return
)

2023-09-08 20:02:18 [info     ] dataset info                   dataset_info=DatasetInfo(observation_signature=Signature(dtype=[dtype('float64')], shape=[(20,)]), action_signature=Signature(dtype=[dtype('float64')], shape=[(3,)]), reward_signature=Signature(dtype=[dtype('float64')], shape=[(1,)]), action_space=<ActionSpace.CONTINUOUS: 1>, action_size=3)
2023-09-08 20:02:18 [info     ] Directory is created at d3rlpy_logs/DecisionTransformer_20230908200218
2023-09-08 20:02:18 [warning  ] Skip building models since they're already built.
2023-09-08 20:02:18 [info     ] Parameters                     params={'observation_shape': [20], 'action_size': 3, 'config': {'type': 'decision_transformer', 'params': {'batch_size': 64, 'gamma': 0.99, 'observation_scaler': {'type': 'none', 'params': {}}, 'action_scaler': {'type': 'none', 'params': {}}, 'reward_scaler': {'type': 'none', 'params': {}}, 'context_size': 25, 'learning_rate': 1e-06, 'encoder_factory': {'type': 'default', 'params': {'activation

Epoch 1/7:   0%|          | 0/6000 [00:00<?, ?it/s]

2023-09-08 20:06:35 [info     ] DecisionTransformer_20230908200218: epoch=1 step=6000 epoch=1 metrics={'time_sample_batch': 0.004876475811004638, 'time_algorithm_update': 0.026909297704696654, 'loss': 0.00017994437613742776, 'time_step': 0.031905724128087364, 'environment': -208.79112648672307} step=6000
2023-09-08 20:06:35 [info     ] Model parameters are saved to d3rlpy_logs/DecisionTransformer_20230908200218/model_6000.d3


Epoch 2/7:   0%|          | 0/6000 [00:00<?, ?it/s]

2023-09-08 20:10:51 [info     ] DecisionTransformer_20230908200218: epoch=2 step=12000 epoch=2 metrics={'time_sample_batch': 0.005058402101198832, 'time_algorithm_update': 0.026965004563331604, 'loss': 0.00014311955041678933, 'time_step': 0.032143854896227515, 'environment': -133.98751810369998} step=12000
2023-09-08 20:10:51 [info     ] Model parameters are saved to d3rlpy_logs/DecisionTransformer_20230908200218/model_12000.d3


...

2023-09-08 20:23:52 [info     ] DecisionTransformer_20230908200218: epoch=5 step=30000 epoch=5 metrics={'time_sample_batch': 0.005596179326375326, 'time_algorithm_update': 0.028198011636734008, 'loss': 8.70608066549418e-05, 'time_step': 0.03394583837191264, 'environment': -210.5076988379282} step=30000
2023-09-08 20:23:52 [info     ] Model parameters are saved to d3rlpy_logs/DecisionTransformer_20230908200218/model_30000.d3


Epoch 6/7:   0%|          | 0/6000 [00:00<?, ?it/s]

2023-09-08 20:28:04 [info     ] DecisionTransformer_20230908200218: epoch=6 step=36000 epoch=6 metrics={'time_sample_batch': 0.004649069190025329, 'time_algorithm_update': 0.026724875768025715, 'loss': 7.6033546348602e-05, 'time_step': 0.03150016419092814, 'environment': -180.37120120160643} step=36000
2023-09-08 20:28:04 [info     ] Model parameters are saved to d3rlpy_logs/DecisionTransformer_20230908200218/model_36000.d3


Epoch 7/7:   0%|          | 0/6000 [00:00<?, ?it/s]

2023-09-08 20:32:16 [info     ] DecisionTransformer_20230908200218: epoch=7 step=42000 epoch=7 metrics={'time_sample_batch': 0.004718012332916259, 'time_algorithm_update': 0.026709372719128925, 'loss': 6.672062979244705e-05, 'time_step': 0.03154738159974416, 'environment': -135.43053234128877} step=42000
2023-09-08 20:32:16 [info     ] Model parameters are saved to d3rlpy_logs/DecisionTransformer_20230908200218/model_42000.d3
